In [1]:
import getpass
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

In [2]:
os.environ["OPENAI_API_KEY"] = getpass.getpass()

In [3]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4")

In [4]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


# This is a runnable. This is something similar to a chain
with_message_history = RunnableWithMessageHistory(model, get_session_history)

In [5]:
# This session id is for a single chat session
# Everytime a user starts a new conversation, it is like a new session
config = {"configurable": {"session_id": "abc2"}}

In [6]:
from langchain_core.messages import HumanMessage

response = with_message_history.invoke(
    [HumanMessage(content="Hi! I am bob")], config=config
)

In [7]:
response.content

'Hello Bob! How can I assist you today?'

In [8]:
response = with_message_history.invoke(
    [HumanMessage(content="What is my name?")], config=config
)

In [9]:
response.content

'Your name is Bob.'

In [10]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [11]:
response = chain.invoke({"messages": [HumanMessage(content="Hi! I am bob")]})

In [12]:
response.content

'Hello, Bob! How can I assist you today?'

In [13]:
with_message_history = RunnableWithMessageHistory(chain, get_session_history)

In [14]:
config = {"configurable": {"session_id": "abc5"}}

response = with_message_history.invoke(
    [HumanMessage(content="Hi! I'm Jim")],
    config=config,
)
response.content

'Hello, Jim! How can I assist you today?'

In [15]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'Your name is Jim.'

In [16]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [17]:
# Now you have added the language tag in this
response = chain.invoke(
    {"messages": [HumanMessage(content="hi! I'm bob")], "language": "Spanish"}
)

response.content

'¡Hola! Soy tu asistente. ¿Cómo puedo ayudarte hoy, Bob?'

In [18]:
# Can I have a mlutilingual chatbot with history
# I will talk in English and let it talk in different languages
with_message_history = RunnableWithMessageHistory(
    chain, get_session_history, input_messages_key="messages"
)

In [19]:
config = {"configurable": {"session_id": "abc11"}}

In [20]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="hi! I'm todd")], "language": "Spanish"},
    config=config,
)

response.content

'¡Hola! Soy tu asistente. ¿Cómo puedo ayudarte hoy, Todd?'

In [21]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="What is my name")], "language": "Italian"},
    config=config,
)
response.content

'Il tuo nome è Todd.'